In [1]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import time
import pandas as pd

ctx= ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [2]:
def getStockList(text_):
    stockList = list()
    url = "https://money.cnn.com/data/hotstocks/"
    html = urllib.request.urlopen(url, context=ctx).read()
    soup = BeautifulSoup(html, "lxml")
    header = soup.find('h3',text=text_)
    table = header.find_next_sibling('table')
    rows = table.find_all('tr')
    for tr in rows:
        if tr.find('td') != None:
            td = tr.find('td')
            ticker_symbol = td.a.text
            ticker_link = td.a.get('href')
            title = td.span.text
            stockList.append((ticker_symbol,title,"https://money.cnn.com/"+ticker_link))
    return stockList

In [3]:
def getStockInfo(stockList):
    stockInfoList = []
    for stock in stockList:
        stockDetails = []
        url = stock[2]
        html = urllib.request.urlopen(url, context=ctx).read()
        soup = BeautifulSoup(html, "lxml")
        header = soup.find('h3',text="Today’s Trading")
        table = header.find_next_sibling()
        rows = table.find_all('tr')
        for row in rows:
            columns = row.find_all('td')
            for col in columns:
                stockDetails.append(col.text)
        stockInfoList.append((stock[0],stock[1],stockDetails[0],stockDetails[1],stockDetails[2],stockDetails[3],stockDetails[6],stockDetails[7],stockDetails[10],stockDetails[11]))
    return stockInfoList

In [4]:
mostActiveStocks = getStockList("Most Actives")
gainerStocks = getStockList("Gainers")
losersStocks = getStockList("Losers")

In [5]:
mostActiveStocksInfo = getStockInfo(mostActiveStocks)
gainerStocksInfo = getStockInfo(gainerStocks)
losersStocksInfo = getStockInfo(losersStocks)

In [6]:
#creating a dataframe to store all the details regarding stocks
df = pd.DataFrame(columns = ['Category', 'CompanyCode', 'CompanyName','Open','PreviousClose','Volume','MarketCap'])
#iterating through each active stock details and appending to dataframe
for content in mostActiveStocksInfo:
    df = df.append({'Category':'Most Actives', 'CompanyCode':content[0], 'CompanyName':content[1], 'Open':content[5], 'PreviousClose':content[3], 'Volume':content[7], 'MarketCap':content[9]}, 
                ignore_index = True)
#iterating through each gainer stock details and appending to dataframe
for content in gainerStocksInfo:
    df = df.append({'Category':'Gainers', 'CompanyCode':content[0], 'CompanyName':content[1], 'Open':content[5], 'PreviousClose':content[3], 'Volume':content[7], 'MarketCap':content[9]}, 
                ignore_index = True)
#iterating through each loser stock details and appending to dataframe
for content in losersStocksInfo:
    df = df.append({'Category':'Losers', 'CompanyCode':content[0], 'CompanyName':content[1], 'Open':content[5], 'PreviousClose':content[3], 'Volume':content[7], 'MarketCap':content[9]}, 
                ignore_index = True)

In [7]:
#printing the dataframe
df

,Category,CompanyCode,CompanyName,Open,PreviousClose,Volume,MarketCap
0,Most Actives,GE,General Electric Co,12.80,12.82,"40,405,028",$116.8B
1,Most Actives,F,Ford Motor Co,11.55,11.33,"36,948,660",$46.3B
2,Most Actives,BAC,Bank of America Corp,40.86,41.18,"24,085,559",$361.5B
3,Most Actives,FCX,Freeport-McMoRan Inc,44.19,42.75,"23,244,889",$64.4B
4,Most Actives,T,AT&T Inc,32.16,31.97,"16,059,075",$229.6B
5,Most Actives,XOM,Exxon Mobil Corp,59.01,60.04,"14,915,112",$264.3B
6,Most Actives,CCL,Carnival Corp,24.97,24.78,"14,707,045",$26.1B
7,Most Actives,PFE,Pfizer Inc,39.50,39.69,"14,084,841",$220.8B
8,Most Actives,OXY,Occidental Petroleum Corp,24.63,25.12,"12,547,086",$25.6B
9,Most Actives,WFC,Wells Fargo & Co,45.39,45.64,"11,838,805",$192.4B


In [8]:
#getting the current time for name of file
fileName = time.strftime("%Y%m%d-%H%M%S"+".csv")
#conevrting the dataframe and save it csv file
df.to_csv(fileName, index=False)